# **[Tensorflow] Doc2vec**

### **1. Data loading**

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import math
from konlpy.tag import Komoran

In [ ]:
train_df = pd.read_csv('./input/ratings_train.txt', delimiter='\t')
pos_df = train_df[train_df['label'] == 1]['document']
neg_df = train_df[train_df['label'] == 0]['document']

### **2. Data preprocessing**
#### **(1) Text를 word의 seq로 변경**

In [ ]:
komoran = Komoran() 

In [ ]:
pos_data = []
pos_label = []
for pos in pos_df:
    try:
        words = komoran.nouns(pos)
        # 단어가 3개 이상 등장한 review 에 대해서만 데이터 set을 생성한다.
        if len(words) > 3:
            pos_data.append(words)
            # 긍정을 의미하는 label 이다.
            pos_label.append(1)
    except:
        pass

In [ ]:
neg_data = []
neg_label = []
for neg in neg_df:
    try:
        words = komoran.nouns(neg)
        if len(words) > 3:
            neg_data.append(words)
            neg_label.append(0)
    except:
        pass

In [ ]:
texts = pos_data + neg_data
labels = pos_label + neg_label

print(len(texts))
print(len(labels))

In [ ]:
texts[:5]

#### **(2) Word의 seq를 number의 seq로 변경 (word2vec의 input : 정수)**

In [ ]:
# indexing dictionary
words = []
for text in texts:
    words.extend(text)

In [ ]:
from collections import Counter
voca_size = 25000
corpus = {}

# 빈도수가 높은 단어 순서대로, indexing이 된다.
for word, freq in Counter(words).most_common(voca_size):
    corpus[word] = len(corpus)

# 결과 디버깅을 위하여, number seq에서, number에 대응하는 word를 찾기 위한 lookup dictionary
corpus_rev = dict(zip(corpus.values(), corpus.keys()))

In [ ]:
# Number seq를 생성한다. 즉 단어에 대응되는 num의 seq로 바꾸는 작업
num_seqs = []

for text in texts:
    num_seq = []
    for word in text:
        if word in corpus:
            idx = corpus[word]
        else:
            idx = 0  # 없는 단어는 0으로 한다. voca_size를 지정했기 때문이다.
        num_seq.append(idx)
    num_seqs.append(num_seq)

In [ ]:
num_seqs[0]

#### **(3) Input batch (source context words + doc id, target word)의 tuple를 생성**
 - Doc2vec은 document id를 source context words에 추가한다.
 - 학습 모델은 source context words + document id -> target word가 된다.

In [ ]:
# Seqs : [src_ctx_word1, src_ctx_word2, target], doc_id
# Inputs : [[src_ctx_word1, src_ctx_word2, doc_id], [target]]
# Batch : [ src_ctx_word1 (+) src_ctx_word2 , doc_id ], label : [ target ]
def generate_batch_data(num_seqs, batch_size, window_size):
    # Batch 데이터
    batch_data = []
    label_data = []
    
    while len(batch_data) < batch_size:
        # Batch 데이터를 생성하고자 하는 문서의 문서 번호(문서 내에서의 순서 index)
        random_seq_idx = int(np.random.choice(len(num_seqs), size=1))
        random_seq = num_seqs[random_seq_idx]
        
        sources_and_target = []
        for idx in range(window_size, len(random_seq)):
            sources = random_seq[idx-window_size : idx]
            target = random_seq[idx]
            sources_and_target.append((sources, target))
        # Target word, source context word를 분리한다.
        sources_list, target_list = [list(x) for x in zip(*sources_and_target)]
        # 문서 번호 추가
        sources_list = [srcs + [random_seq_idx] for srcs in sources_list]
        # 데이터를 생성한다.
        batch_data.extend(sources_list)
        label_data.extend(target_list)
                
    # 마지막 iteration에서 batch size를 초과할 수 있으므로, batch 크기만 생성
    batch_data = batch_data[:batch_size]
    label_data = label_data[:batch_size]
    
    # Placeholder에 feeding하기 위해서는 numpy array 형태로 변환한다.
    batch_data = np.array(batch_data)
    # Nce_loss에서 사용하기 위해서 shape을 변경한다.
    label_data = np.transpose(np.array([label_data]))
    
    return (batch_data, label_data)

In [ ]:
# Validation set을 생성한다. 생성한 embedding matrix와의 거리를 측정할 수 있다.
# 이것은 단순히 similarity를 보여주기 위한 예이다.
valid_words = ['한국', '사랑', '스릴러', '장르']
valid_words = [corpus[elem] for elem in valid_words]
valid_datasets = tf.constant(valid_words, dtype=tf.int32)

### **3. Modeling using tensorflow**

In [ ]:
embedding_size = 128
doc_embedding_size = 128
concat_size = embedding_size + doc_embedding_size
batch_size = 128
window_size = 3  # How many subnets to consider left and right
num_sampled = 64  # Number of negative examples to sample
learning_rate = 0.001
num_steps = 50000  # Tensorflow tutorial
num_docs = len(num_seqs)

# Input data
with tf.name_scope('inputs'):
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size, window_size + 1])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])

"""
Embedding -> Train input -> embed
[[1,2,3], -> [0,2]       -> [[1,2,3],
 [4,5,6],                    [7,8,9]]
 [7,8,9]]
"""
with tf.device('/GPU:0'):
    with tf.name_scope('embeddings'):
        embeddings = tf.Variable(tf.random_uniform([voca_size, embedding_size], -1, 1))
        doc_embeddings = tf.Variable(tf.random_uniform([num_docs, doc_embedding_size], -1.0, 1.0))
        # Source context word의 embedding 합
        embed = tf.zeros([batch_size, embedding_size])
        for idx in range(window_size):
            embed += tf.nn.embedding_lookup(embeddings, train_inputs[:, idx])
        # Doc indices
        doc_indices = tf.slice(train_inputs, [0, window_size], [batch_size, 1])
        # Doc embedding
        doc_embed = tf.nn.embedding_lookup(doc_embeddings, doc_indices)
        # Source context word의 embedding을 더한 값과, doc의 embedding을 concat
        final_embed = tf.concat(axis=1, values=[embed, tf.squeeze(doc_embed)])

    # Weights
    with tf.name_scope('weights'):  
        nce_weights = tf.Variable(tf.truncated_normal([voca_size, concat_size],
                                              stddev = 1.0 / np.sqrt(concat_size)))
    # Biases
    with tf.name_scope('biases'):
        nce_biases = tf.Variable(tf.zeros([voca_size]))

# Loss
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.nce_loss(
                weights=nce_weights,
                biases=nce_biases,
                labels=train_labels,
                inputs=final_embed,
                num_sampled=num_sampled,
                num_classes=voca_size))

# Optimizer
with tf.name_scope('optimizer'):
    optimizer = tf.train.AdamOptimizer().minimize(loss)

In [ ]:
# 검증 단어 집합과 임베딩된 모든 단어 사이의 코사인 유사도를 구해보자.
norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
normalized_embeddings = embeddings / norm
# 검증 단어 집합의 embedding을 구한다.
valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_datasets)
similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

In [ ]:
# session 초기화
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
for i in range(num_steps):
    batch_inputs, batch_labels = generate_batch_data(num_seqs, batch_size, window_size)
    feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}
    sess.run(optimizer, feed_dict=feed_dict)
    if i % 100 == 0:
        print("=================================")
        loss_val = sess.run(loss, feed_dict=feed_dict)
        # Githup에 올릴때는 주석을 한다. 결과를 보고 싶으면 주석을 해제하면 된다.
        print('Loss at step {}: {}'.format(i, loss_val))
        
        # Valid word와의 similarity를 확인하여 검증을 해보자.
        sim = sess.run(similarity, feed_dict=feed_dict)
        for j in range(len(valid_words)):
            valid_word = corpus_rev[valid_words[j]]
            top_k = 5
            nearest = (-sim[j, :]).argsort()[1:top_k+1]
            log_str = 'Nearest to {}:'.format(valid_word)
            for k in range(top_k):
                close_word = corpus_rev[nearest[k]]
                log_str = '%s %s,' % (log_str, close_word)
            # Githup에 올릴때 주석, 결과를 보고 싶으면 주석을 해제하면 된다.
            print(log_str)

In [ ]:
sess.close()